<div style="text-align: left"> Virro, H., Jemeljanova, M., Chan, W.T., Kmoch, A., Uuemaa, E. </div>
<div style="text-align: left"> Department of Geography, University of Tartu </div>
<div style="text-align: left"> <a href="https://landscape-geoinformatics.ut.ee/">https://landscape-geoinformatics.ut.ee/</a> </div>

<h1><center>Spatial modelling and interpretability with Random Forest</center></h1>
<h3><center>AGILE 2024 CONFERENCE</center></h3>
<h3><center>University of Glasgow, June 6 2024</center></h3>

Machine learning has been increasingly used due to its capabilities to work with large amounts of data, while having minimal assumptions on shape or distribution of variables. However, machine-learning models are often considered a **black-box**, and a lack of interpretability would mean that it is hard to determine if the model has found meaningful and realistic relationships between different phenomena. Thus, **there is a strong need to be able to explain and interpret machine-learning models** to understand the effects and relationships of the underlying modelled processes and used covariates.

Recently, **various methods to interpret the relationships** between the covariates and the target variable in machine learning **have been introduced** (see, e.g, Molnar 2024). **In this workshop, we will present such methods**, namely **partial dependence plots** and **Shapley values**, and provide tips on how to make use of these methods to build less biased, better understandable, and more robust models. This workshop will provide hands-on tasks on interpreting a machine learning model’s results using the Python programming language.

In this workshop, we will use an example of predicting the **total nitrogen** (mg/L) using various environmental variables (topography, soil, land use and land cover, hydrology, agriculture, and climate) from various data sources (see the details in Virro et al. 2022). The total nitrogen data was obtained from the KESE (Estonian Environmental Agency, 2021) environment monitoring system website maintained by the Estonian Environment Agency and was the average value between years 2016 and 2020. This workshop is adapted from the study "Random forest-based modeling of stream nutrients at national level in a data-scarce region" authored by Virro et al. (https://doi.org/10.1016/j.scitotenv.2022.156613).

We will be using Colab in this workshop, however all the materials, including the .yml file needed to create the environment, are available on the GitHub repository: https://github.com/LandscapeGeoinformatics/agile2024_rf_interpretability

### TOC for Jupyter notebook (does not work for Colab):
* [0. Install required packages](#install_package)
* [1. Dependences/ Library Import](#import_library)
* [2. Prepare RF training and test sets](#Data_preparation)
* [3. Hyperparameter tuning](#HPO)
* [4. Model Training (with model parameters based on Hyperparameter tuning)](#HPO_Train)
* [5. Calculate SHAP values](#shap_values)
* [6. Reduce the number of features based on SHAP values](#reduce_model)
* [7. SHAP analysis of the new model](#reduce_model_shap)

# 0. Install required packages <a class="anchor" id="install_package"></a>

First, we will install the required packages.

In [ ]:
# Get the requirement file from github repo
!wget -c https://raw.githubusercontent.com/LandscapeGeoinformatics/agile2024_rf_interpretability/main/workshop_materials/requirements.txt
!pip install -r requirements.txt

# 1. Dependences/ Library Import <a class="anchor" id="import_library"></a>
Let's import the packages. For the data processing, we will use **numpy** and **pandas**, to read in geospatial data, **geopandas** will be used. **scikit-learn** will be used for Random Forest modelling, **shap** for shap value calculation and plotting.The partial dependence plots will be generated with scikit-learn's **PartialDependenceDisplay** function. **Seaborn** and **matplotlib** will be used for plotting.

In [ ]:
# Import packages
import seaborn as sns
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
from sklearn.metrics import r2_score
import shap
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
sns.set_theme()

Random Forest relies on randomisation (as the name implies) in subsetting data for decision trees. We set the seed so that each re-run of the Random Forest models results in the same random state and thus, the same outcome.

In [ ]:
# Set seed
random_seed = 0
random_state = np.random.seed(random_seed)
random.seed(random_seed)

# 2. Prepare RF training and test sets <a class="anchor" id="Data_preparation"></a>

The geopackage (*gpkg*) with the prediction target (total nitrogen) and feature values (land use, soil etc) is located in a GitHub repository. Let's download it directly from there.

`wget` is a command line function that is used to retrieve data from web servers. We use exclamation mark in front to open a shell and close it immediatelly after the function is executed.

In [ ]:
# Getting the gpkg from repo
!wget -c https://github.com/LandscapeGeoinformatics/agile2024_rf_interpretability/raw/main/workshop_materials/agile2024_tn_mean_obs_sites.gpkg

## 2.1 Data Exploration (very simple one)
Let's explore the contents of the file. We will use the **geopandas** package, since has the capacity to read in and work with spatial data.

In [ ]:
# Read observation data
fp = "agile2024_tn_mean_obs_sites.gpkg"
obs_data = gpd.read_file(fp)
display(obs_data)

As we can see, the file contains a site code (site_code) and an id (obs_id) for each sampling point, the prediction target as total nitrogen (mg/l)
(obs_value), as well as 82 features, and the geometry, where the spatial information is stored.

In [ ]:
obs_data.info()

## 2.2 Data visualization
Let's plot the data spatially, using the observed value of total nitrogen (mg/L).

In [ ]:
# Create interactive plot of observation values
obs_data.explore(
    column="obs_value",
    cmap="YlOrRd",
    tooltip=["site_code", "obs_value"],
    marker_kwds={"radius": 4},
    style_kwds={"color": "black", "weight": 1, "fillOpacity": 0.9}
)

## 2.3 Training and Testing Dataset preparation
Now, we will extract the features (that will be used to predict the target) and the prediction target in seperate variables. From the training data, we will remove the id columns the prediction target (first three columns) and the geometry column (the last column), as they are not necessary in the model training. We will seperately save the prediction target values. **Throughout the workshop, we will use X to represent covariates (or features), and Y as the target variable.**

In [ ]:
# Extract features and target
X = obs_data.iloc[:, 3:-1]
y = obs_data["obs_value"]

To test the model generalizability, we will split the dataset randomly into 70% of training data and 30% testing data. We will immediatelly create four different dataframes for training and testing to ensure that the features correspond to the respective prediction targets.

In [ ]:
# Split the data into training and test sets
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

We will train our model on 167 samples.

In [ ]:
X_train

In [ ]:
y_train

# 3. Hyperparameter tuning <a class="anchor" id="HPO"></a>

Usually, the default parameters provided in the scikit-learn package yield acceptable results. However, depending on the data, a substantially higher performance can be achieved with hyperparameter optimisation and hyperparameter tuning is **very important to avoid overfitting**. It is recommended to start with the automatic methods (e.g., grid search or random search) and then manually check some parameters.

Today, we will use the RandomisedSearchCV (Bergstra and Bengio, 2012) algorithm. We will define a list of possible hyperparameter values (value ranges), and the algorithm will determine which combination of them performs the best.

We will perform 100 runs (`n_iter=100`), and the best hyperparameter combination will be the one with the smallest resulting mean squared error (MSE) when running a model.

We will tune the following parameters:

* `n_estimators` (number of trees in random forest);

* `max_depth` (maximum number of levels in a tree)

In [ ]:
# Search for hyperparameters
def search_hyperparams(X, y, random_state):

    # Number of trees in random forest - test between 50 to 200 with testing interval of 25
    step = 25
    n_estimators = list(np.arange(start=50, stop=200 + step, step=step))

    # Maximum number of levels in a tree - test between 5 and 20 with testing interval of 5
    step = 5
    max_depth = list(np.arange(start=5, stop=20 + step, step=step))

    # Create dictionary from parameters
    param_distributions = {
        "n_estimators": n_estimators,
        "max_depth": max_depth
    }

    # Perform search for hyperparameters
    estimator = RandomForestRegressor()
    rf_random = RandomizedSearchCV(
        estimator=estimator, param_distributions=param_distributions, n_iter=100, verbose=2, random_state=random_state,
        n_jobs=-1
    )
    rf_random.fit(X, y)

    # Get best parameters
    params = rf_random.best_params_
    params["oob_score"] = True

    return params

In [ ]:
%%time

# Perform search for hyperparameters
params = search_hyperparams(X_train, y_train, random_state)

In [ ]:
# Show the best hyperparameters
params

# 4. Model Training (with model parameters based on Hyperparameter tuning) <a class="anchor" id="HPO_Train"></a>

Now that we have the optimal hyperparameters, let's train our model using them. First, we define the algorithm we will use (Random Forest) and set the hyperparameters.

In [ ]:
# RF regressor
regressor = RandomForestRegressor()

In [ ]:
# Set hyperparameters
regressor = RandomForestRegressor(**params)

Next, we train the model on our covariate and prediction target data.

In [ ]:
# Fit model
regressor.fit(X_train, y_train)

Let's see how well our model performed on the training data by calculating the R squared value. As R squared values span [0;1], where 1 means an ideal fit between the observations and predicted values, and the value goes down as the model predication get worse.

In [ ]:
# Calculate accuracy on training set
regressor.score(X_train, y_train)

In [ ]:
# Predict
Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)

Let's also determine the model generalisation capacity by determining R squared on independent data that the model has not seen (i.e., our test data). While the score is fair, it is lower than for the training data, meaning that there has been a fair bit of overfitting to the training data.

In [ ]:
# Calculate accuracy on test set
r2_score(y_test, Y_test_pred)

# 5. Calculate SHAP values <a class="anchor" id="shap_values"></a>

SHAP (SHapely Additive exPlanations) is a method that originally comes from cooperative game theory, where it was used to determine each player's contribution on the resulting game score. Similarly, in machine learning, it is used to quantify the contribution of each covariate on the prediction target in an additive, linear manner (meaning that the contributions of features are summed). The SHAP are used to reveal if the contribution is positive or negative, and to quantify it. The values are in the units of the prediction target (in our case, mg/L), which makes them easy to interpret. Read more about SHAP in Molnar 2024 and https://shap.readthedocs.io/en/latest/.

The SHAP value method is model-agnostic, meaning that it can be used on various machine learning algorithms. For our Random Forest models, we will use the TreeSHAP (function **TreeExplainer**), which is a variant of SHAP made specifically for decision tree models.

In [ ]:
# Calculate SHAP values
explainer = shap.TreeExplainer(regressor)
shap_values = explainer.shap_values(X_train)

Shap values are organised in an array of arrays by sample and then by the covariate.

In [ ]:
shap_values

The SHAP values can be calculated both globally (for the whole dataset) and locally (distinguishing individual instances). First, let's plot SHAP globally using the summary plot.

This plot shows the **absolute contribution of each feature**. The features are ordered based on their importance, and here, 20 out of 82 most important features are plotted. On average, the highest contribution to the total nitrogen value is by the proportion of arable land in catchment (arable_prop), features mean rock content of the first layer (rock1_mean), silt % of the first soil layer (silt1_mean) and rip_veg_nat_prop (total area of riparian vegetation buffer around natural streams divided by catchment area), all exceeding 0.1 mg/L. All other features contributed by 0.1 mg/L or less.

**NB!** While SHAP might seem similar to permutation based feature importance, the latter describes feature contribution to the prediction accuracy, not to the target value! Please be mindful when interpreting the results.

In [ ]:
# SHAP summary bar plot
shap.summary_plot(shap_values=shap_values, features=X_train, feature_names=X_train.columns, plot_type="bar")
plt.tight_layout()

For a more thorough exploration, it is always advisable to plot the **summary beeswarm plot**, where individual instances can be observed. The interpretation of the summary beeswarm plot is as follows: each point is the SHAP value in a distinct sampling location. For each feature, there are as many points as sampling locations in our dataset. The color describes the covariate value relative to the range of covariate values in the dataset, for each covariate seperately. The red dots represent high values (for example, high rock content in the first layer), while blue represents low values.

The impact on the target value (in our case, total nitrogen mg/L) can be read from the x axis: the more a certain point is positioned to the right, the higher the contribution, and vice versa; for example, high arable proportion in the catchment (arable_prop) results in up to 0.6 mg/L higher total nitrogen concentration in certain locations.

Besides that, we can see that features like rock content in the first layer (rock1_mean) and the arable proportion in the catchment (arable_prop) contribute positively to the target, since higher feature values result in higher contribution, while features like k1_mean (mean hydraulic conductivity of the first layer) and rip_veg_nat_prop (total area of riparian vegetation buffer around natural streams divided by catchment area) have a negative influence: lower values result in higher total nitrogen concentration.

As you can see, the contribution of high vs. low values of feature is not symmetric. For example, low values of rip_veg_nat_prop contribute up to 2.6 mg/L increase on the target, while high values of the same feature decrease the total nitrogen content by only 0.6 mg/L at most.

**SHAP values and partial dependence plots (below) are a great tool of model validation: the modeler can see if the relationships revealed by the model are logical and correspond to the domain knowledge.**

In [ ]:
# SHAP summary beeswarm plot
shap.summary_plot(shap_values=shap_values, features=X_train, feature_names=X_train.columns)
plt.tight_layout()

To understand better how each predictor is contributing to the model and how it is related to the target variable, we can use partial dependence plots. We don't want to plot all 82 features and therefore we will subset the 10 most important covariates based on SHAP values.

## 5.1 Top 10 features with the highest SHAP value

In [ ]:
#extracting top 10 features based on SHAP values
feature_names = X_train.columns
rf_resultX = pd.DataFrame(shap_values, columns = feature_names)
vals = np.abs(rf_resultX.values).mean(0)
shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=["feature","feature_importance_vals"])
shap_importance.sort_values(by=["feature_importance_vals"],
                               ascending=False, inplace=True)
shap_importance[:10]

## 5.2 Plot the partial dependence

Partial Dependence plots show the marginal effect of feature on a target in a ML model, that is, the contribution of a specific feature on the target after the average impacts of other features have been removed. Usually, partial dependence plots are drawn for one to two features at time, as the interpretation of more features becomes increasingly complex.

With these plots, one can determine if the influence of feature on the prediction target is linear or non-linear. Moreover, the x axis shows the values of the respective feature, while the y axis visualises the target values (in our case, total nitrogen, mg/L). Thus, these plots are intuitive to understand and interpret and certain thresholds of influence can be determined. For example, with an increasing proportion of arable land (feature "arable_prop"), the concentration of total nitrogen increases. A steep increase is noted when the arable proportion exceeds 40%, raising from around 2.5 mg/L to around 4.5 mg/L. As with SHAP, these plots are a great validation tool. New insights can also be gained, however caution should be exercised when interpreting the relationships. For example, keep in mind that the marks of data indicate value density around certain values in your dataset, therefore a lower certainty is in the data range with low number of points (e.g., range 20-40 vs. range 60-80 for the feature k1_mean).

In [ ]:
# Create partial dependence plot (Full Model)
fig, ax = plt.subplots(figsize=(12, 12))
disp = PartialDependenceDisplay.from_estimator(regressor, X_train, shap_importance[:10]["feature"].values, ax=ax)
fig.subplots_adjust(hspace=0.3)
fig.tight_layout()

# 6. Reduce the number of features based on SHAP values <a class="anchor" id="reduce_model"></a>

As a general rule, we are interested in models needing as little features as possible due to computational etc. limitations. There are various feature reduction methods, such as permutation based feature importance, forward feature selection, and others. Now we will decrease our feature set using the results of SHAP and will explore the difference in the prediction accuracy.

We will use the 10 most influential features (from the original 82) as determined by SHAP and will re-train the Random Forest model with this decreased feature set.

In [ ]:
abs_mean_shap_df = shap_importance[:10]

In [ ]:
# List of most important features
n_features = 10
top_features = abs_mean_shap_df["feature"].head(n_features).to_list()
print(top_features)

We subset the relevant features from the original training dataset.

In [ ]:
# Generate new training and test feature sets
X_train_reduced = X_train[top_features]
X_test_reduced = X_test[top_features]

## 6.1 Train model on reduced data

Again, we will determine the optimal hyperparameters with the exact same methodology as before, but on the reduced training set.

In [ ]:
%%time

# Perform search for hyperparameters
params_reduced = search_hyperparams(X_train_reduced, y_train, random_state)

Let's train the model with the optimal hyperparameters and the reduced training set the same way we did before.

In [ ]:
# RF regressor
regressor_reduced = RandomForestRegressor()

# Set hyperparameters
regressor_reduced.set_params(**params_reduced)

# Fit model
regressor_reduced.fit(X_train_reduced, y_train)

As you can see, the accuracy on the training and test set with the lower feature set has remained practically the same with less data needed. Great! Keep in mind though, that this is not a guarantee- the scores can decrease, depending on the data, etc.

In [ ]:
# Calculate accuracy on training set
regressor_reduced.score(X_train_reduced, y_train)

In [ ]:
# Predict
Y_train_pred_reduced = regressor_reduced.predict(X_train_reduced)
Y_test_pred_reduced = regressor_reduced.predict(X_test_reduced)

In [ ]:
# Calculate accuracy on test set
r2_score(y_test, Y_test_pred_reduced)

# 7. SHAP analysis of the new model <a class="anchor" id="reduce_model_shap"></a>

Now, we will again plot the SHAP values and explore the differences in contribution between the full and the decrease data sets.

In [ ]:
# Calculate SHAP values
explainer_reduced = shap.TreeExplainer(regressor_reduced)
shap_values_reduced = shap.TreeExplainer(regressor_reduced).shap_values(X_train_reduced)

The order has remained the same but the influence has increased a bit. That is a good indicator, showing that the model is quite robust and you have meaningful predictors.

In [ ]:
# SHAP summary beeswarm plot
shap.summary_plot(shap_values=shap_values_reduced, features=X_train_reduced, feature_names=X_train_reduced.columns)
plt.tight_layout()

Let's look at the partial dependence plots once again. As before, the relationships remain the same. As with SHAP, these plots are a great validation tool. New insights can also be gained, however caution should be exercised when interpreting the relationships.

In [ ]:
# Create partial dependence plot
fig, ax = plt.subplots(figsize=(12, 12))
disp = PartialDependenceDisplay.from_estimator(regressor_reduced, X_train_reduced, X_train_reduced.columns, ax=ax)
fig.subplots_adjust(hspace=0.3)
fig.tight_layout()

## 7.1 Plot the predictions spatially

Now, we will plot all predictions spatially (for both train and test locations) of our target value (total nitrogen, mg/L) from our model with the reduced feature set. First, we will create a dataframe with all locations.

In [ ]:
# Concatenate reduced features and sort by index
X_reduced = pd.concat([X_train_reduced, X_test_reduced]).sort_index()
X_reduced.head()

Next, we will use our trained model to predict the target values for all locations.

In [ ]:
# Predict with the reduced model
predictions = regressor_reduced.predict(X_reduced)
display(predictions)

In [ ]:
# Add predictions to observation data
obs_data_pred = obs_data.copy()
obs_data_pred["pred_value"] = predictions
obs_data_pred.head()

Now, we will plot our predicted values.

In [ ]:
# Create interactive plot of predicted values
obs_data_pred.explore(
    column="pred_value",
    cmap="YlOrRd",
    tooltip=["site_code", "obs_value", "pred_value"],
    marker_kwds={"radius": 4},
    style_kwds={"color": "black", "weight": 1, "fillOpacity": 0.9}
)

**Residuals** are the difference between the observed and predicted values. If residuals are spatially autocorrelated (meaning that values in close vicinity are similar), it indicates that there might be a feature missing. This information can be used to further improve the model.

In [ ]:
# Calculate residuals between observed and predicted values
obs_data_pred["residual"] = obs_data_pred["obs_value"] - obs_data_pred["pred_value"]

In [ ]:
# Create interactive plot of residuals
obs_data_pred.explore(
    column="residual",
    cmap="RdBu_r",
    tooltip=["site_code", "obs_value", "pred_value", "residual"],
    marker_kwds={"radius": 4},
    style_kwds={"color": "black", "weight": 1, "fillOpacity": 0.9}
)

Next, let's explore the observed and predicted values. The plot indicates that the model overestimated the some of the low values and underestimated some of the high values, which is characteristic of Random Forest models. It could be due to the characteristics of how data is subsampled for the decision trees, meaning that the extreme values are not always included in training each tree, therefore the model does not learn all the possible value combinations.

In [ ]:
# Plot observed and predicted values
fig, ax = plt.subplots(1, 1)
sns.scatterplot(data=obs_data_pred, x="obs_value", y="pred_value")
plt.tight_layout()